In [1]:
import numpy as np #for numerical computations
import pandas as pd #for dataframe operations

from matplotlib import pyplot as plt #for viewing images and plots
%matplotlib inline 
#So that Matplotlib plots don't open in separate windows outside the notebook

import urllib #For fetching data from Web URLs

import cv2   #For image processing

from sklearn.preprocessing import LabelEncoder    #For encoding categorical variables
from sklearn.model_selection import train_test_split #For splitting of dataset

#All tensorflow utilities for creating, training and working with a CNN
import tensorflow as tf 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import layers 
from tensorflow.keras import Model 
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

**Dataset Hand**

In [8]:
objek = 'hand'

In [2]:
def prepare_data(file_name):
    df = pd.read_csv(r'{}'.format(file_name), names=["path", "target"])
    print(df["path"].head())
    np.where(df.applymap(lambda x: x == ''))
    
    # X Input Data
    image_list = []

    for path in df['path'] :
#         print(path) 
        image_arr = cv2.imread(path)
        image_224x224 = cv2.resize(image_arr, (150, 150))
        image_list.append(image_224x224.astype('float')) 
    X = np.array(image_list)
    print("Total Data = ", X.shape)

    # Y Output Data
    encoder = LabelEncoder()
    Targets = encoder.fit_transform(df['target'])
    n_classes = 2
    Y = to_categorical(Targets, num_classes = n_classes)
    
    return X, Y

In [10]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

0    MURA-v1.1/train/XR_HAND/patient10169/study1_ne...
1    MURA-v1.1/train/XR_HAND/patient10121/study1_po...
2    MURA-v1.1/train/XR_HAND/patient09111/study1_ne...
3    MURA-v1.1/train/XR_HAND/patient11008/study1_ne...
4    MURA-v1.1/train/XR_HAND/patient09780/study1_po...
Name: path, dtype: object
Total Data =  (5543, 150, 150, 3)
Data Train (4711, 150, 150, 3)
Data Val (832, 150, 150, 3)


In [10]:
# resnet_model = Sequential()
# resnet_model.add(ResNet50(include_top=False, weights='imagenet', pooling='avg'))
# resnet_model.add(Dense(2, activation='softmax'))

# resnet_model.compile(optimizer = 'adamax', loss = 'binary_crossentropy', metrics = ['acc'])

In [3]:
resnet_model = ResNet50(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

for layer in resnet_model.layers:
    layer.trainable = False

x = layers.Flatten()(resnet_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(2, activation='softmax')(x)

model = tf.keras.models.Model(resnet_model.input, x)

model.compile(optimizer = Adamax(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

In [13]:
save_at = "ResNet_Model/resnet_{}.h5".format(objek)
save_best = ModelCheckpoint (filepath = save_at, monitor='val_acc', save_best_only=True, save_freq='epoch', save_weights_only=False, mode='max')
hist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20) 

Epoch 1/20
148/148 [==============================] - 260s 2s/step - loss: 0.6049 - acc: 0.7285 - val_loss: 0.5682 - val_acc: 0.7428
Epoch 2/20
148/148 [==============================] - 252s 2s/step - loss: 0.5857 - acc: 0.7302 - val_loss: 0.5680 - val_acc: 0.7428
Epoch 3/20
148/148 [==============================] - 258s 2s/step - loss: 0.5840 - acc: 0.7304 - val_loss: 0.5755 - val_acc: 0.7428
Epoch 4/20
148/148 [==============================] - 252s 2s/step - loss: 0.5809 - acc: 0.7304 - val_loss: 0.5690 - val_acc: 0.7428
Epoch 5/20
148/148 [==============================] - 237s 2s/step - loss: 0.5801 - acc: 0.7304 - val_loss: 0.5651 - val_acc: 0.7428
Epoch 6/20
148/148 [==============================] - 228s 2s/step - loss: 0.5768 - acc: 0.7304 - val_loss: 0.5806 - val_acc: 0.7428
Epoch 7/20
148/148 [==============================] - 228s 2s/step - loss: 0.5789 - acc: 0.7304 - val_loss: 0.5651 - val_acc: 0.7428
Epoch 8/20
148/148 [==============================] - 228s 2s/step - 

**Dataset Humerus**

In [14]:
objek = 'humerus'

In [15]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

0    MURA-v1.1/train/XR_HUMERUS/patient02717/study1...
1    MURA-v1.1/train/XR_HUMERUS/patient02922/study1...
2    MURA-v1.1/train/XR_HUMERUS/patient02746/study1...
3    MURA-v1.1/train/XR_HUMERUS/patient02917/study1...
4    MURA-v1.1/train/XR_HUMERUS/patient02739/study1...
Name: path, dtype: object
Total Data =  (1272, 150, 150, 3)
Data Train (1081, 150, 150, 3)
Data Val (191, 150, 150, 3)


In [16]:
save_at = "ResNet_Model/resnet_{}.h5".format(objek)
save_best = ModelCheckpoint (filepath = save_at, monitor='val_acc', save_best_only=True, save_freq='epoch', save_weights_only=False, mode='max')
hist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20) 

Epoch 1/20
34/34 [==============================] - 57s 2s/step - loss: 0.7121 - acc: 0.5199 - val_loss: 0.6955 - val_acc: 0.4869
Epoch 2/20
34/34 [==============================] - 54s 2s/step - loss: 0.6902 - acc: 0.5282 - val_loss: 0.6979 - val_acc: 0.4869
Epoch 3/20
34/34 [==============================] - 59s 2s/step - loss: 0.6884 - acc: 0.5532 - val_loss: 0.6971 - val_acc: 0.4974
Epoch 4/20
34/34 [==============================] - 55s 2s/step - loss: 0.6887 - acc: 0.5365 - val_loss: 0.6951 - val_acc: 0.4974
Epoch 5/20
34/34 [==============================] - 59s 2s/step - loss: 0.6886 - acc: 0.5301 - val_loss: 0.6922 - val_acc: 0.5079
Epoch 6/20
34/34 [==============================] - 53s 2s/step - loss: 0.6868 - acc: 0.5560 - val_loss: 0.6929 - val_acc: 0.5026
Epoch 7/20
34/34 [==============================] - 55s 2s/step - loss: 0.6840 - acc: 0.5486 - val_loss: 0.6942 - val_acc: 0.4974
Epoch 8/20
34/34 [==============================] - 56s 2s/step - loss: 0.6824 - acc: 0.55

**Dataset Shoulder**

In [2]:
objek = 'shoulder'

In [5]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

0    MURA-v1.1/train/XR_SHOULDER/patient01685/study...
1    MURA-v1.1/train/XR_SHOULDER/patient02206/study...
2    MURA-v1.1/train/XR_SHOULDER/patient01976/study...
3    MURA-v1.1/train/XR_SHOULDER/patient01071/study...
4    MURA-v1.1/train/XR_SHOULDER/patient02526/study...
Name: path, dtype: object
Total Data =  (8364, 150, 150, 3)
Data Train (7109, 150, 150, 3)
Data Val (1255, 150, 150, 3)


In [6]:
save_at = "ResNet_Model/resnet_{}.h5".format(objek)
save_best = ModelCheckpoint (filepath = save_at, monitor='val_acc', save_best_only=True, save_freq='epoch', save_weights_only=False, mode='max')
hist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20) 

Epoch 1/20
223/223 [==============================] - 382s 2s/step - loss: 0.7117 - acc: 0.5092 - val_loss: 0.6857 - val_acc: 0.5586
Epoch 2/20
223/223 [==============================] - 354s 2s/step - loss: 0.6932 - acc: 0.5275 - val_loss: 0.6960 - val_acc: 0.5171
Epoch 3/20
223/223 [==============================] - 363s 2s/step - loss: 0.6909 - acc: 0.5279 - val_loss: 0.6855 - val_acc: 0.5641
Epoch 4/20
223/223 [==============================] - 367s 2s/step - loss: 0.6832 - acc: 0.5501 - val_loss: 0.6828 - val_acc: 0.5689
Epoch 5/20
223/223 [==============================] - 342s 2s/step - loss: 0.6815 - acc: 0.5555 - val_loss: 0.6832 - val_acc: 0.5649
Epoch 6/20
223/223 [==============================] - 375s 2s/step - loss: 0.6803 - acc: 0.5639 - val_loss: 0.6829 - val_acc: 0.5633
Epoch 7/20
223/223 [==============================] - 446s 2s/step - loss: 0.6818 - acc: 0.5500 - val_loss: 0.6809 - val_acc: 0.5618
Epoch 8/20
223/223 [==============================] - 360s 2s/step - 

**Dataset Wrist**

In [4]:
objek = 'wrist'

In [5]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

0    MURA-v1.1/train/XR_WRIST/patient07688/study1_n...
1    MURA-v1.1/train/XR_WRIST/patient06904/study1_p...
2    MURA-v1.1/train/XR_WRIST/patient06806/study1_p...
3    MURA-v1.1/train/XR_WRIST/patient06442/study1_p...
4    MURA-v1.1/train/XR_WRIST/patient00305/study1_n...
Name: path, dtype: object
Total Data =  (9739, 150, 150, 3)
Data Train (8278, 150, 150, 3)
Data Val (1461, 150, 150, 3)


In [6]:
save_at = "ResNet_Model/resnet_{}.h5".format(objek)
save_best = ModelCheckpoint (filepath = save_at, monitor='val_acc', save_best_only=True, save_freq='epoch', save_weights_only=False, mode='max')
hist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20) 

Epoch 1/20
259/259 [==============================] - 449s 2s/step - loss: 0.6788 - acc: 0.5931 - val_loss: 0.6501 - val_acc: 0.5927
Epoch 2/20
259/259 [==============================] - 420s 2s/step - loss: 0.6484 - acc: 0.6214 - val_loss: 0.6453 - val_acc: 0.6099
Epoch 3/20
259/259 [==============================] - 488s 2s/step - loss: 0.6459 - acc: 0.6249 - val_loss: 0.6550 - val_acc: 0.5962
Epoch 4/20
259/259 [==============================] - 649s 3s/step - loss: 0.6393 - acc: 0.6340 - val_loss: 0.6380 - val_acc: 0.6304
Epoch 5/20
259/259 [==============================] - 684s 3s/step - loss: 0.6390 - acc: 0.6322 - val_loss: 0.6446 - val_acc: 0.6078
Epoch 6/20
259/259 [==============================] - 690s 3s/step - loss: 0.6372 - acc: 0.6346 - val_loss: 0.6355 - val_acc: 0.6407
Epoch 7/20
259/259 [==============================] - 1258s 5s/step - loss: 0.6352 - acc: 0.6378 - val_loss: 0.6331 - val_acc: 0.6386
Epoch 8/20
259/259 [==============================] - 704s 3s/step -

**Dataset Elbow**

In [ ]:
objek = 'elbow'

In [ ]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

In [ ]:
save_at = "ResNet_Model/resnet_{}.h5".format(objek)
save_best = ModelCheckpoint (filepath = save_at, monitor='val_acc', save_best_only=True, save_freq='epoch', save_weights_only=False, mode='max')
hist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20) 

**Dataset Finger**

In [ ]:
objek = 'finger'

In [ ]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

In [ ]:
save_at = "ResNet_Model/resnet_{}.h5".format(objek)
save_best = ModelCheckpoint (filepath = save_at, monitor='val_acc', save_best_only=True, save_freq='epoch', save_weights_only=False, mode='max')
hist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20) 

**Dataset Forearm**

In [ ]:
objek = 'forearm'

In [ ]:
X, Y = prepare_data("{}_with_label_randomized.csv".format(objek))
X = X/255
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13)
print("Data Train", X_train.shape)
print("Data Val", X_val.shape)

In [ ]:
save_at = "ResNet_Model/resnet_{}.h5".format(objek)
save_best = ModelCheckpoint (filepath = save_at, monitor='val_acc', save_best_only=True, save_freq='epoch', save_weights_only=False, mode='max')
hist = model.fit(X_train, Y_train, validation_data = (X_val, Y_val), callbacks=[save_best], epochs = 20) 